# ModelNet40 Data Analysis

Author: Deeepwin  
Date: 26.08.2022 
***

In [27]:
import os
import pandas as pd
import numpy as np

import torch
from data import load_data_cls

## Load ModelNet40 Data

For data preparation, download [ModelNet40](https://modelnet.cs.princeton.edu/#) and store files in the following folder: `data/modelnet40_ply_hdf5_2048` 



In [28]:
train_data, train_label = load_data_cls('train')
train_data.shape, train_label.shape


((9840, 2048, 3), (9840, 1))

In [29]:
test_data, test_label = load_data_cls('test')
test_data.shape, test_label.shape

((2468, 2048, 3), (2468, 1))

## Viszalize an Object

In [30]:
# select object
object_idx  = 1

In [31]:
import open3d as o3d
import plotly.graph_objects as go

In [32]:
def draw(geometries):
    graph_obj = []

    for gm in geometries:
        geometry_type = gm.get_geometry_type()
        
        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            pts = np.asarray(gm.points)
            clr = None  #for colors
            if gm.has_colors():
                clr = np.asarray(gm.colors)
            elif gm.has_normals():
                clr = (0.5, 0.5, 0.5) + np.asarray(gm.normals) * 0.5
            else:
                gm.paint_uniform_color((1.0, 0.0, 0.0))
                clr = np.asarray(gm.colors)

            sc = go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2], mode='markers', marker=dict(size=1, color=clr))
            graph_obj.append(sc)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            tri = np.asarray(gm.triangles)
            vert = np.asarray(gm.vertices)
            clr = None
            if gm.has_triangle_normals():
                clr = (0.5, 0.5, 0.5) + np.asarray(gm.triangle_normals) * 0.5
                clr = tuple(map(tuple, clr))
            else:
                clr = (1.0, 0.0, 0.0)
            
            mesh = go.Mesh3d(x=vert[:,0], y=vert[:,1], z=vert[:,2], i=tri[:,0], j=tri[:,1], k=tri[:,2], facecolor=clr, opacity=0.50)
            graph_obj.append(mesh)
        
    fig = go.Figure(
        data=graph_obj,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()

In [33]:

# convert numpy array to point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(test_data[object_idx])

In [34]:
# get data statistics
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df_train = pd.DataFrame(np.array(train_data).flatten(), columns=['train'])
df_test = pd.DataFrame(np.array(test_data).flatten(), columns=['test'])
df_train.describe(), df_test.describe()

(             train
 count 60456960.000
 mean        -0.000
 std          0.341
 min         -1.000
 25%         -0.212
 50%          0.000
 75%          0.217
 max          1.000,
               test
 count 15163392.000
 mean        -0.000
 std          0.343
 min         -1.000
 25%         -0.218
 50%         -0.000
 75%          0.219
 max          1.000)

In [35]:
# replace drawing function to show in notebook
o3d.visualization.draw_geometries = draw 
o3d.visualization.draw_geometries([pcd])